In [1]:
# common paths
BASE = "/content/work"; STEP = "03_patches"  # cambia per file
import os, json; os.makedirs(f"{BASE}/{STEP}", exist_ok=True)
CFG_PATH = f"{BASE}/config.json"
CFG = json.load(open(CFG_PATH)) if os.path.exists(CFG_PATH) else {}
IN_DIR  = f"{BASE}/02_pt"       # lo step da cui leggi
OUT_DIR = f"{BASE}/{STEP}"      # dove scrivi

In [ ]:
!pip install pydicom SimpleITK

In [ ]:
pip install rt-utils


In [ ]:
#debug

import pydicom

def leggi_series_description_da_rtstruct(rtstruct_path):
    try:
        ds = pydicom.dcmread(rtstruct_path, stop_before_pixels=True)
        modality = getattr(ds, "Modality", "")
        if modality != "RTSTRUCT":
            print(f"❌ Il file non è un RTSTRUCT (Modality={modality})")
            return
        desc = getattr(ds, "SeriesDescription", None)
        if desc:
            print(f"📄 SeriesDescription: '{desc}'")
        else:
            print("⚠️ Il campo 'SeriesDescription' non è presente nel file.")
    except Exception as e:
        print(f"❌ Errore nella lettura del file: {e}")

# ESEMPIO D'USO
# Inserisci il path del file RTSTRUCT:
rtstruct_path = "/content/drive/MyDrive/NeuroOnco/Sorgente/ID1/ID1_ANON48282_RTst_2025-02-03_154251_RM.ENCEFALO..CON.MDC_OP1_n1__00004/2.16.840.1.114362.1.12359305.26387206364.706733084.641.21.dcm"
leggi_series_description_da_rtstruct(rtstruct_path)


❌ Errore nella lettura del file: [Errno 2] No such file or directory: '/content/drive/MyDrive/NeuroOnco/Sorgente/ID1/ID1_ANON48282_RTst_2025-02-03_154251_RM.ENCEFALO..CON.MDC_OP1_n1__00004/2.16.840.1.114362.1.12359305.26387206364.706733084.641.21.dcm'


In [ ]:
#path_studio = "/content/drive/MyDrive/NeuroOnco/Sorgente/ROSSI"



In [ ]:
# -*- coding: utf-8 -*-
"""
Modulo completo per:
- Analisi geometrica serie DICOM
- Costruzione volume 4D multiparametrico
- Estrazione e integrazione di strutture RTSTRUCT
- Visualizzazione interattiva con overlay
"""

import os
import time
import json
import numpy as np
import torch
import pydicom
import SimpleITK as sitk
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from skimage import measure
import cv2

In [ ]:
# ----------------------------
# UTILITY: ANALISI SERIE DICOM
# ----------------------------
def trova_serie_valide(path_base):
    serie = []
    for root, _, files in os.walk(path_base):
        dcm_files = [f for f in files if f.lower().endswith(".dcm")]
        if len(dcm_files) < 2:
            continue
        try:
            dcm = pydicom.dcmread(os.path.join(root, dcm_files[0]), stop_before_pixels=True)
            if dcm.Modality == "RTSTRUCT":
                continue
            desc = getattr(dcm, "SeriesDescription", "N/A")
            serie.append((desc, root))
        except:
            continue
    return serie

def ordina_dicom(path_serie):
    dicom_files = [f for f in os.listdir(path_serie) if f.lower().endswith(".dcm")]
    paths = [os.path.join(path_serie, f) for f in dicom_files]
    dcm_list = []

    for f in paths:
        try:
            dcm = pydicom.dcmread(f, stop_before_pixels=True)
            # ✅ Controlla che sia un'immagine con geometria
            if hasattr(dcm, "ImagePositionPatient") and hasattr(dcm, "ImageOrientationPatient"):
                dcm_list.append((dcm, f))
        except:
            continue

    if not dcm_list:
        raise ValueError(f"❌ Nessun DICOM con geometria trovata in {path_serie}")

    try:
        orient = np.array(dcm_list[0][0].ImageOrientationPatient).reshape(2, 3)
    except Exception as e:
        raise ValueError(f"❌ Errore leggendo geometria nel primo file valido: {e}")

    normale = np.cross(orient[0], orient[1])
    proiezioni = [(np.dot(np.array(d.ImagePositionPatient), normale), f) for d, f in dcm_list]
    paths_ordinati = [f for _, f in sorted(proiezioni)]
    return paths_ordinati


In [ ]:
# ----------------------------
# COSTRUZIONE VOLUME 4D
# ----------------------------
def carica_volume(paths_dicom):
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(paths_dicom)
    image = reader.Execute()
    return sitk.Cast(image, sitk.sitkFloat32)

def normalizza_array(img):
    array = sitk.GetArrayFromImage(img).astype(np.float32)
    array[array == fill_value] = np.nan
    p1, p99 = np.nanpercentile(array, [1, 99])
    array = np.clip((array - p1) / (p99 - p1 + 1e-5), 0, 1)
    return array

def resample_con_default(img, reference):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(np.nan)
    return resampler.Execute(img)


# Costruzione volume 4D con eventuali strutture direttamente integrate
# ----------------------------


def costruisci_volume_4d_solo_immagini(path_base, descrizione_riferimento="csT1W_3D MDC"):
    serie = trova_serie_valide(path_base)
    print(f"Trovate {len(serie)} serie DICOM valide")
    volumi, nomi, img_rif = [], [], None
    path_rif = None  # ✅ inizializzata

    for desc, path in serie:
        if descrizione_riferimento.lower() in desc.lower():
            print(f"✅ Serie di riferimento trovata: {desc}")
            paths_ordinati = ordina_dicom(path)
            img_rif = carica_volume(paths_ordinati)
            volumi.append(normalizza_array(img_rif, fill_value))
            nomi.append(desc)
            path_rif = path  # ✅ assegnata qui
            break

    if img_rif is None or path_rif is None:
        raise ValueError(f"❌ Serie di riferimento '{descrizione_riferimento}' non trovata.")

    for desc, path in serie:
        if descrizione_riferimento.lower() in desc.lower():
            continue
        print(f"📦 Serie compatibile trovata: {desc}")
        paths_ordinati = ordina_dicom(path)
        img = carica_volume(paths_ordinati)
        img_resamp = resample_con_default(img, img_rif, fill_value)
        volumi.append(normalizza_array(img_resamp, fill_value))
        nomi.append(desc)

    vol4d = np.stack(volumi, axis=-1)
    print(f"✅ Volume 4D costruito: shape = {vol4d.shape}")
    return vol4d, nomi, img_rif, path_rif


In [ ]:
# ----------------------------
# RTSTRUCT: Estrazione e Integrazione ROI
# ----------------------------
def estrai_maschera_da_rtstruct(rtstruct_path, img_rif):
    rt = pydicom.dcmread(rtstruct_path)
    structures = rt.StructureSetROISequence
    rois = [roi.ROIName for roi in structures]
    if not rois:
        raise ValueError("❌ Nessuna ROI trovata nel RTSTRUCT.")
    roi_idx = structures[0].ROINumber
    contours = [roi for roi in rt.ROIContourSequence if roi.ReferencedROINumber == roi_idx]
    if not contours:
        raise ValueError("❌ Nessun contour trovato per la ROI selezionata.")

    mask = sitk.Image(img_rif.GetSize(), sitk.sitkUInt8)
    mask.SetSpacing(img_rif.GetSpacing())
    mask.SetOrigin(img_rif.GetOrigin())
    mask.SetDirection(img_rif.GetDirection())

    for contour in contours[0].ContourSequence:
        if len(contour.ContourData) < 6:
            continue
        pts = np.array(contour.ContourData).reshape(-1, 3)
        slice_z = pts[0, 2]
        slice_index = int(round((slice_z - mask.GetOrigin()[2]) / mask.GetSpacing()[2]))
        img2d = np.zeros((mask.GetSize()[1], mask.GetSize()[0]), dtype=np.uint8)
        poly_pts = [(int(round((x - mask.GetOrigin()[0]) / mask.GetSpacing()[0])),
                     int(round((y - mask.GetOrigin()[1]) / mask.GetSpacing()[1]))) for x, y, z in pts]
        cv2.fillPoly(img2d, [np.array(poly_pts, dtype=np.int32)], 1)
        arr3d = sitk.GetArrayFromImage(mask)
        if 0 <= slice_index < arr3d.shape[0]:
            arr3d[slice_index, :, :] = np.maximum(arr3d[slice_index, :, :], img2d)
        mask = sitk.GetImageFromArray(arr3d)
        mask.CopyInformation(img_rif)

    return mask

def integra_strutture_risamplate_come_serie(vol4d, nomi_serie, rtstruct_path, path_studio, griglie_serie):
    rt = pydicom.dcmread(rtstruct_path)
    roi_map = {}
    for roi in rt.ROIContourSequence:
        if not hasattr(roi, "ContourSequence") or len(roi.ContourSequence) == 0:
            continue
        if len(roi.ContourSequence[0].ContourData) < 9:
            continue
        roi_num = roi.ReferencedROINumber
        roi_name = next((r.ROIName for r in rt.StructureSetROISequence if r.ROINumber == roi_num), f"ROI_{roi_num}")
        if hasattr(roi.ContourSequence[0], "ContourImageSequence"):
            uid = roi.ContourSequence[0].ContourImageSequence[0].ReferencedSOPInstanceUID
            roi_map[roi_name] = uid

    sop_to_path, sop_to_seriesdesc = {}, {}
    for root, _, files in os.walk(path_studio):
        for f in files:
            if f.lower().endswith(".dcm"):
                try:
                    path = os.path.join(root, f)
                    dcm = pydicom.dcmread(path, stop_before_pixels=True)
                    if hasattr(dcm, "SOPInstanceUID"):
                        sop_to_path[dcm.SOPInstanceUID] = path
                        sop_to_seriesdesc[dcm.SOPInstanceUID] = getattr(dcm, "SeriesDescription", "Unknown")
                except:
                    continue

    immagini_caricate = {}
    for roi_name, uid in roi_map.items():
        if uid not in sop_to_path:
            print(f"⚠️ ROI '{roi_name}' ignorata: UID non trovato.")
            continue
        desc = sop_to_seriesdesc[uid]
        if desc not in griglie_serie:
            print(f"⚠️ ROI '{roi_name}' ignorata: griglia non trovata per la serie '{desc}'.")
            continue
        path_img = sop_to_path[uid]
        path_serie = os.path.dirname(path_img)
        if path_serie not in immagini_caricate:
            files_dcm = sorted([os.path.join(path_serie, f) for f in os.listdir(path_serie) if f.endswith(".dcm")])
            img_base = sitk.ReadImage(files_dcm)
            immagini_caricate[path_serie] = img_base
        else:
            img_base = immagini_caricate[path_serie]
        print(f"📌 ROI '{roi_name}' → serie '{desc}'")
        mask = estrai_maschera_da_rtstruct(rtstruct_path, img_base)
        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(griglie_serie[desc])
        resampler.SetTransform(sitk.Transform(3, sitk.sitkIdentity))
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        resampler.SetDefaultPixelValue(0)
        resampler.SetOutputPixelType(sitk.sitkUInt8)
        mask_risamp = resampler.Execute(mask)
        mask_np = sitk.GetArrayFromImage(mask_risamp)
        if mask_np.shape == vol4d.shape[:3]:
            vol4d = np.concatenate([vol4d, mask_np[..., np.newaxis]], axis=-1)
            nomi_serie.append(f"MASK_{roi_name}")
            print(f"✅ ROI '{roi_name}' aggiunta al volume.")
        else:
            print(f"❌ Shape incompatibile per ROI '{roi_name}', saltata.")

    print("\n📊 Riepilogo integrazione ROI:")
    print(f"🔢 ROI totali trovate nel RTSTRUCT: {len(roi_map)}")
    print(f"✅ ROI aggiunte al volume: {len([n for n in nomi_serie if n.startswith('MASK_')])}")
    print(f"❌ ROI saltate: {len(roi_map) - len([n for n in nomi_serie if n.startswith('MASK_')])}")
    print(f"📚 ROI nel volume: {[n for n in nomi_serie if n.startswith('MASK_')]}")

    return vol4d, nomi_serie

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from IPython.display import display

def visualizza_volume4d_con_istogramma(volume4d, nomi_serie=None):
    n_serie = volume4d.shape[0]
    n_slices = volume4d.shape[1]

    if nomi_serie is None:
        nomi_serie = [f"Serie {i}" for i in range(n_serie)]

    def aggiorna(z_idx, serie_idx):
        slice_img = volume4d[serie_idx, z_idx, :, :]  # ✅ GIUSTO


        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        axs[0].imshow(slice_img, cmap='gray')
        axs[0].set_title(f"{nomi_serie[serie_idx]}")
        axs[0].axis('off')

        axs[1].hist(slice_img.ravel(), bins=50, color='gray')
        axs[1].set_title("Istogramma intensità")
        axs[1].set_xlabel("Valore")
        axs[1].set_ylabel("Frequenza")

        plt.tight_layout()
        plt.show()

    slider_z = widgets.IntSlider(min=0, max=n_slices-1, step=1, value=n_slices//2, description='Slice Z')
    slider_s = widgets.IntSlider(min=0, max=n_serie-1, step=1, value=0, description='Serie')

    widgets.interact(aggiorna, z_idx=slider_z, serie_idx=slider_s)


In [ ]:
#da eseguire in debug

#vol4d, nomi_serie, img_rif, path_rif = costruisci_volume_4d_solo_immagini(path_studio)


In [ ]:
#visualizza_volume4d_con_istogramma(vol4d, nomi_serie)


STRUTTURE

In [ ]:
# ----------------------------
# UTILITY: ANALISI SERIE DICOM
# ----------------------------
def trova_serie_valide(path_base):
    serie = []
    for root, _, files in os.walk(path_base):
        dcm_files = [f for f in files if f.lower().endswith(".dcm")]
        if len(dcm_files) < 2:
            continue
        try:
            dcm = pydicom.dcmread(os.path.join(root, dcm_files[0]), stop_before_pixels=True)
            if dcm.Modality == "RTSTRUCT":
                continue
            desc = getattr(dcm, "SeriesDescription", "N/A")
            serie.append((desc, root))
        except:
            continue
    return serie

def ordina_dicom(path_serie):
    dicom_files = [f for f in os.listdir(path_serie) if f.lower().endswith(".dcm")]
    paths = [os.path.join(path_serie, f) for f in dicom_files]
    dcm_list = []

    for f in paths:
        try:
            dcm = pydicom.dcmread(f, stop_before_pixels=True)
            if hasattr(dcm, "ImagePositionPatient") and hasattr(dcm, "ImageOrientationPatient"):
                dcm_list.append((dcm, f))
        except:
            continue

    if not dcm_list:
        raise ValueError(f"❌ Nessun DICOM con geometria trovata in {path_serie}")

    orient = np.array(dcm_list[0][0].ImageOrientationPatient).reshape(2, 3)
    normale = np.cross(orient[0], orient[1])
    proiezioni = [(np.dot(np.array(d.ImagePositionPatient), normale), f) for d, f in dcm_list]
    paths_ordinati = [f for _, f in sorted(proiezioni)]
    return paths_ordinati


In [ ]:
# ----------------------------
# COSTRUZIONE VOLUME 4D
# ----------------------------
def carica_volume(paths_dicom):
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(paths_dicom)
    image = reader.Execute()
    return sitk.Cast(image, sitk.sitkFloat32)

def normalizza_array(img):
    array = sitk.GetArrayFromImage(img).astype(np.float32)
    array[array == fill_value] = np.nan
    p1, p99 = np.nanpercentile(array, [1, 99])
    array = np.clip((array - p1) / (p99 - p1 + 1e-5), 0, 1)
    return array

def resample_con_default(img, reference, default_value=-1000.0):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(default_value)
    return resampler.Execute(img)

def costruisci_volume_4d_solo_immagini(path_base, descrizione_riferimento="csT1W_3D MDC"):
    """
    Costruisce un volume 4D multiparametrico da una directory DICOM.
    La prima serie usata è quella che corrisponde alla descrizione di riferimento.

    Ritorna:
        - vol4d: volume 4D normalizzato (z, y, x, serie)
        - nomi: nomi delle serie usate
        - img_rif: immagine SimpleITK della serie di riferimento
        - path_rif: path alla serie DICOM di riferimento (usato per RTSTRUCT)
    """
    serie = trova_serie_valide(path_base)
    print(f"Trovate {len(serie)} serie DICOM valide")
    volumi, nomi, img_rif = [], [], None
    path_rif = None  # <- nuova variabile

    for desc, path in serie:
        if descrizione_riferimento.lower() in desc.lower():
            print(f"✅ Serie di riferimento trovata: {desc}")
            paths_ordinati = ordina_dicom(path)
            img_rif = carica_volume(paths_ordinati)
            volumi.append(normalizza_array(img_rif, fill_value))
            nomi.append(desc)
            path_rif = path  # <- salviamo il path
            break

    if img_rif is None or path_rif is None:
        raise ValueError(f"❌ Serie di riferimento '{descrizione_riferimento}' non trovata.")

    for desc, path in serie:
        if descrizione_riferimento.lower() in desc.lower():
            continue
        print(f"📦 Serie compatibile trovata: {desc}")
        paths_ordinati = ordina_dicom(path)
        img = carica_volume(paths_ordinati)
        img_resamp = resample_con_default(img, img_rif, fill_value)
        volumi.append(normalizza_array(img_resamp, fill_value))
        nomi.append(desc)

    vol4d = np.stack(volumi, axis=-1)
    print(f"✅ Volume 4D costruito: shape = {vol4d.shape}")
    return vol4d, nomi, img_rif, path_rif


In [ ]:
def visualizza_volume_con_bordo(volume4d, mask_mdc, nomi_serie=None):
    import matplotlib.pyplot as plt
    import ipywidgets as widgets
    from IPython.display import display
    import numpy as np

    n_slices = volume4d.shape[0]
    n_serie = volume4d.shape[3]

    if nomi_serie is None:
        nomi_serie = [f"Serie {i}" for i in range(n_serie)]

    def aggiorna(z_idx, serie_idx):
        slice_img = volume4d[serie_idx, z_idx, :, :]  # ✅ GIUSTO
        slice_mask = mask_mdc[z_idx, :, :].astype(bool)

        fig, axs = plt.subplots(1, 2, figsize=(12, 6))

        # --- Visualizzazione slice + contorno
        axs[0].imshow(slice_img, cmap='gray')
        axs[0].imshow(slice_mask, cmap='Reds', alpha=0.3)
        axs[0].contour(slice_mask, colors='red', linewidths=1)
        axs[0].set_title(f"{nomi_serie[serie_idx]} con bordo MDC")
        axs[0].axis('off')

        # --- Istogramma sull'intero volume
        vol_img = volume4d[:, :, :, serie_idx]
        vol_mask = mask_mdc.astype(bool)

        inside_values = vol_img[vol_mask]
        outside_values = vol_img[~vol_mask]

        # Escludi valori 0 e 1
        inside_values = inside_values[(inside_values != 0) & (inside_values != 1)]
        outside_values = outside_values[(outside_values != 0) & (outside_values != 1)]

        axs[1].hist(outside_values.ravel(), bins=100, color='gray', alpha=0.6, label='Esterni', density=True)
        axs[1].hist(inside_values.ravel(), bins=100, color='red', alpha=0.6, label='Interni', density=True)
        axs[1].set_title("Istogramma su tutto il volume (normalizzato)")
        axs[1].set_xlabel("Valore normalizzato")
        axs[1].set_ylabel("Densità")
        axs[1].legend()

        plt.tight_layout()
        plt.show()

    slider_z = widgets.IntSlider(min=0, max=n_slices - 1, step=1, value=n_slices // 2, description='Slice Z')
    slider_s = widgets.IntSlider(min=0, max=n_serie - 1, step=1, value=0, description='Serie')

    widgets.interact(aggiorna, z_idx=slider_z, serie_idx=slider_s)



In [ ]:
def riallinea_maschera_rtutils(mask_np, reference_img):
    """
    Riallinea una maschera binaria (z, y, x) proveniente da RTUtils
    sulla griglia della serie DICOM di riferimento (SimpleITK Image)
    """
    import SimpleITK as sitk

    # 1️⃣ Crea immagine dalla maschera binaria (z, y, x)
    mask_img = sitk.GetImageFromArray(mask_np.astype(np.uint8))  # ⬅️ Ricorda: SimpleITK usa (z,y,x)

    # 2️⃣ Imposta geometria della maschera come il riferimento
    mask_img.SetSpacing(reference_img.GetSpacing())
    mask_img.SetOrigin(reference_img.GetOrigin())
    mask_img.SetDirection(reference_img.GetDirection())

    # 3️⃣ Applica resampling coerente
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference_img)
    resampler.SetTransform(sitk.Transform(3, sitk.sitkIdentity))  # Trasformazione identità
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetDefaultPixelValue(0)
    resampler.SetOutputPixelType(sitk.sitkUInt8)

    mask_resampled = resampler.Execute(mask_img)

    # 4️⃣ Ritorna la maschera come array numpy (z, y, x)
    return sitk.GetArrayFromImage(mask_resampled)


In [ ]:
def salva_paziente_standardizzato(path_studio, path_strutture_list, path_output=None, dtype=np.float32):
    import os
    import time
    import numpy as np
    import torch
    import pydicom
    import SimpleITK as sitk
    from rt_utils import RTStructBuilder

    if path_output is None:
        nome_studio = os.path.basename(path_studio.rstrip("/"))
        base_dir = os.path.abspath(os.path.join(path_studio, "..", ".."))
        output_dir = os.path.join(base_dir, "vol_pt")
        os.makedirs(output_dir, exist_ok=True)
        path_output = os.path.join(output_dir, f"{nome_studio}.pt")

    serie = trova_serie_valide(path_studio)

    print("\n📋 Serie disponibili nel paziente:")
    for desc, _ in serie:
        print(" -", desc)

    serie_keywords = {
        "T1W":   ["t1w", "cst1w", "csT1W"],
        "FLAIR": ["flair", "csflair"],
        "DWI":   ["dwi", "adc", "diff"],
        "KTRAN": ["ktrans", "ktran", "regt1"],
        "CBV":   ["cbv"],
        "APT":   ["apt"]
    }

    ordine_fisso = list(serie_keywords.keys())
    volumi = []
    nomi_serie_finale = []
    img_rif, path_rif = None, None

    for nome_corto in ordine_fisso:
        keywords = serie_keywords[nome_corto]
        print(f"\n🔎 Cerco serie per: {nome_corto}", end="")

        trovata = False
        for desc, path in serie:
            desc_norm = desc.lower().replace(" ", "")
            if any(kw in desc_norm for kw in keywords):
                print(f" --- ✅ Serie {nome_corto} trovata: {desc}")
                paths_ordinati = ordina_dicom(path)
                img = carica_volume(paths_ordinati)

                if img_rif is None:
                    img_rif = img
                    path_rif = path
                    vol = sitk.GetArrayFromImage(img).astype(dtype)
                else:
                    img_resamp = resample_con_default(img, img_rif)
                    vol = sitk.GetArrayFromImage(img_resamp).astype(dtype)

                volumi.append(vol)
                nomi_serie_finale.append(nome_corto)
                trovata = True
                break

        if not trovata:
            print(" --- ⚠️ Non trovata")
            if img_rif is None:
                raise ValueError("❌ Nessuna serie utile trovata per iniziare.")
            shape = img_rif.GetSize()
            vol_vuoto = np.full((shape[2], shape[1], shape[0]), np.nan, dtype=dtype)
            volumi.append(vol_vuoto)
            nomi_serie_finale.append(nome_corto)

    vol4d = np.stack(volumi, axis=0)  # (C, Z, Y, X)

    roi_masks = {}

    # 🔍 Cerca RTSTRUCT con descrizione REF e struttura REF
    rtstruct_ref = None
    for root, _, files in os.walk(path_studio):
        for f in files:
            if f.lower().endswith(".dcm"):
                try:
                    path_dcm = os.path.join(root, f)
                    dcm = pydicom.dcmread(path_dcm, stop_before_pixels=True)
                    if getattr(dcm, "Modality", "") == "RTSTRUCT" and getattr(dcm, "SeriesDescription", "").strip().upper() == "REF":
                        rtstruct_ref = path_dcm
                        break
                except:
                    continue
        if rtstruct_ref:
            break

    if rtstruct_ref:
        print(f"\n📥 RTSTRUCT con SeriesDescription='REF' trovato: {rtstruct_ref}")
        try:
            from rt_utils import RTStructBuilder
            rt_ref = RTStructBuilder.create_from(
                dicom_series_path=path_rif,
                rt_struct_path=rtstruct_ref
            )
            if "REF" in rt_ref.get_roi_names():
                mask_np = rt_ref.get_roi_mask_by_name("REF").astype(np.uint8)
                mask_np = np.transpose(mask_np, (2, 0, 1))  # (Z, Y, X)
                voxel_attivi = int(np.sum(mask_np))

                expected_shape = (img_rif.GetSize()[2], img_rif.GetSize()[1], img_rif.GetSize()[0])
                if mask_np.shape == expected_shape and voxel_attivi > 0:
                    roi_masks["REF"] = torch.tensor(mask_np, dtype=torch.uint8)
                    print(f"✅ ROI 'REF' importata correttamente. Voxel attivi: {voxel_attivi}")
                else:
                    print(f"⚠️ ROI 'REF' trovata ma shape errata o vuota.")
            else:
                print("⚠️ Nessuna ROI chiamata 'REF' trovata nel RTSTRUCT.")
        except Exception as e:
            print(f"❌ Errore durante l'importazione del RTSTRUCT 'REF': {e}")
    else:
        print("⚠️ Nessun RTSTRUCT con SeriesDescription='REF' trovato.")


    for path_strutture in path_strutture_list:
        rtstruct_path = None
        for root, _, files in os.walk(path_strutture):
            for f in files:
                if f.lower().endswith(".dcm"):
                    try:
                        dcm = pydicom.dcmread(os.path.join(root, f), stop_before_pixels=True)
                        if getattr(dcm, "Modality", "") == "RTSTRUCT":
                            rtstruct_path = os.path.join(root, f)
                            break
                    except:
                        continue
            if rtstruct_path:
                break

        if rtstruct_path is None:
            print(f"⚠️ Nessun RTSTRUCT trovato in {path_strutture}")
            continue

        print(f"📂 RTSTRUCT trovato: {rtstruct_path}")
        try:
            rtstruct = RTStructBuilder.create_from(
                dicom_series_path=path_rif,
                rt_struct_path=rtstruct_path
            )
        except Exception as e:
            print(f"❌ Errore durante la creazione dell'RTStruct: {e}")
            continue

        desc_rtstruct = getattr(rtstruct.ds, "SeriesDescription", "RTSTRUCT").replace(" ", "").upper()

        roi_nomi = rtstruct.get_roi_names()
        for nome in roi_nomi:
            try:
                mask_np = rtstruct.get_roi_mask_by_name(nome)
                mask_np = np.transpose(mask_np, (2, 0, 1))  # (Z, Y, X)
                voxel_attivi = int(np.sum(mask_np))

                expected_shape = (img_rif.GetSize()[2], img_rif.GetSize()[1], img_rif.GetSize()[0])
                if mask_np.shape != expected_shape:
                    print(f"⚠️ ROI '{nome}' scartata: shape errata {mask_np.shape}")
                    continue
                if voxel_attivi == 0:
                    print(f"⚠️ ROI '{nome}' vuota ➜ ignorata.")
                    continue

                nome_unico = f"{nome}-{desc_rtstruct}"
                i = 1
                while nome_unico in roi_masks:
                    nome_unico = f"{nome}-{desc_rtstruct}_{i}"
                    i += 1

                roi_masks[nome_unico] = torch.tensor(mask_np.astype(np.uint8), dtype=torch.uint8)
                print(f"✅ ROI '{nome_unico}' integrata. Voxel attivi: {voxel_attivi}")

            except Exception as e:
                print(f"❌ Errore nella ROI '{nome}': {e}")

    data = {
        "volume": torch.tensor(vol4d, dtype=torch.float32 if dtype == np.float32 else torch.float16),
        "roi_masks": roi_masks,
        "nomi_serie": nomi_serie_finale
    }

    start = time.time()
    torch.save(data, path_output)
    duration = time.time() - start
    size_bytes = os.path.getsize(path_output)
    size_mb = size_bytes / (1024 * 1024)
    speed = size_mb / duration if duration > 0 else float('inf')

    print(f"\n✅ File salvato in: {path_output}")
    print(f"📦 Dimensione file: {size_mb:.2f} MB")
    print(f"⏱️ Tempo di scrittura: {duration:.2f} s")
    print(f"🚀 Velocità di salvataggio: {speed:.2f} MB/s")


In [ ]:
#from pprint import pprint
#pprint([desc for desc, _ in trova_serie_valide(path_studio)])


In [ ]:
import os
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
from functools import partial

# 📂 Cartelle
root_dir = "/content/drive/MyDrive/NeuroOnco/Sorgente"
output_dir = "/content/drive/MyDrive/NeuroOnco/Derivate"
log_path = os.path.join(output_dir, "log_derivate.txt")
os.makedirs(output_dir, exist_ok=True)

# 🔍 Trova tutte le cartelle pazienti
tutti_pazienti = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])

# 🔘 Widget per selezione
multi_select = widgets.SelectMultiple(
    options=tutti_pazienti,
    value=tutti_pazienti,  # pre-seleziona tutti
    description='Pazienti:',
    layout=widgets.Layout(width='60%', height='200px')
)

run_button = widgets.Button(description="Esegui elaborazione", button_style='success')

output = widgets.Output()

def esegui_elaborazione(selezionati):
    with open(log_path, "w") as log_file:
        log_file.write(f"🗓️ LOG elaborazione - {datetime.now()}\n")
        log_file.write(f"📁 Sorgente: {root_dir}\n")
        log_file.write(f"📂 Output: {output_dir}\n\n")

        for nome_cartella in selezionati:
            path_studio = os.path.join(root_dir, nome_cartella)
            if not os.path.isdir(path_studio):
                continue

            path_output = os.path.join(output_dir, f"{nome_cartella}.pt")
            log_file.write(f"\n🧾 Paziente: {nome_cartella}\n")

            print(f"\n🚀 Inizio elaborazione: {nome_cartella}")

            try:
                # Inserisci qui la tua funzione personalizzata:
                serie = trova_serie_valide(path_studio)
                serie_disponibili = [desc for desc, _ in serie]
                log_file.write(f"   📋 Serie disponibili: {serie_disponibili}\n")

                serie_attese = {
                    "T1W":   ["t1w", "cst1w", "csT1W"],
                    "FLAIR": ["flair", "csflair"],
                    "DWI":   ["dwi", "adc", "diff"],
                    "KTRAN": ["ktrans", "ktran", "regt1"],
                    "CBV":   ["cbv"],
                    "APT":   ["apt"]
                }

                trovate, mancanti = [], []
                desc_norm_list = [desc.lower().replace(" ", "") for desc in serie_disponibili]

                for nome_corto, keywords in serie_attese.items():
                    trovato = False
                    for desc_norm in desc_norm_list:
                        if any(kw in desc_norm for kw in keywords):
                            trovate.append(nome_corto)
                            trovato = True
                            break
                    if not trovato:
                        mancanti.append(nome_corto)

                log_file.write(f"   ✅ Serie trovate: {trovate}\n")
                log_file.write(f"   ⚠️ Serie mancanti (riempite vuote): {mancanti}\n")

                # Trova tutte le sottocartelle contenenti RTSTRUCT per quel paziente
                path_strutture_list = []
                for subdir in os.listdir(path_studio):
                    path_completo = os.path.join(path_studio, subdir)
                    if os.path.isdir(path_completo):
                        for f in os.listdir(path_completo):
                            if f.lower().endswith(".dcm"):
                                try:
                                    dcm = pydicom.dcmread(os.path.join(path_completo, f), stop_before_pixels=True)
                                    if getattr(dcm, "Modality", "") == "RTSTRUCT":
                                        path_strutture_list.append(path_completo)
                                        break
                                except:
                                    continue

                # Chiamata aggiornata
                salva_paziente_standardizzato(
                    path_studio=path_studio,
                    path_strutture_list=path_strutture_list,
                    path_output=path_output,
                    dtype=np.float16
                )

                log_file.write(f"   💾 Salvato in: {path_output}\n")

                # 📦 Legge il file .pt e logga le ROI
                data = torch.load(path_output)
                roi_masks = data.get("roi_masks", {})
                if roi_masks:
                    roi_nomi = list(roi_masks.keys())
                    log_file.write(f"   🧠 ROI trovate e salvate: {roi_nomi}\n")
                    for nome, mask in roi_masks.items():
                        voxel_attivi = int(mask.sum().item())
                        log_file.write(f"      ➜ {nome}: {voxel_attivi} voxel attivi\n")
                else:
                    log_file.write(f"   ⚠️ Nessuna ROI trovata o salvata.\n")

            except Exception as e:
                log_file.write(f"   ❌ Errore: {str(e)}\n")
                print(f"❌ Errore con paziente {nome_cartella}: {e}")

def on_button_click(b):
    with output:
        output.clear_output()
        selezionati = list(multi_select.value)
        print(f"\n📋 Pazienti selezionati: {selezionati}")
        esegui_elaborazione(selezionati)

run_button.on_click(on_button_click)

# Mostra interfaccia
display(widgets.VBox([multi_select, run_button, output]))


DEBUG
